#### Getting spacecraft positions


In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

import astropy
import sunpy
import astroquery  
from sunpy.coordinates import frames, get_horizons_coord


import heliopy.data.spice as spicedata
import heliopy.spice as spice

import astropy.units as u
import astrospice

/nas/helio/heliofc/miniconda3/envs/helio2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### sunpy - only works for small array sizes

In [2]:
#Solar Orbiter
coord = get_horizons_coord('Solar Orbiter', time={'start': '2022-09-01', 'stop': '2022-09-30', 'step': '60m'})  
solo_heeq = coord.transform_to(frames.HeliographicStonyhurst) #HEEQ
solo_hee = coord.transform_to(frames.HeliocentricEarthEcliptic)  #HEE

solo_time=solo_heeq.obstime.to_datetime()
solo_r=solo_heeq.radius.value
solo_lon=np.deg2rad(solo_heeq.lon.value)
solo_lat=np.deg2rad(solo_heeq.lat.value)


#PSP
coord = get_horizons_coord('Parker Solar Probe', time={'start': '2022-09-01', 'stop': '2022-09-30', 'step': '60m'})  
psp_heeq=coord.transform_to(frames.HeliographicStonyhurst)

psp_time=psp_heeq.obstime.to_datetime()
psp_r=psp_heeq.radius.value
psp_lon=np.deg2rad(psp_heeq.lon.value)
psp_lat=np.deg2rad(psp_heeq.lat.value)

    

ValueError: Input values did not match any of the formats where the format keyword is optional: {'datetime': TypeError('Input values for datetime class must be datetime objects'), 'ymdhms': ValueError("'stop', 'step', 'start' not allowed as YMDHMS key name(s)"), 'iso': TypeError('Input values for iso class must be strings'), 'isot': TypeError('Input values for isot class must be strings'), 'yday': TypeError('Input values for yday class must be strings'), 'datetime64': TypeError('Input values for datetime64 class must be datetime64 objects'), 'fits': TypeError('Input values for fits class must be strings'), 'byear_str': TypeError('Input values for byear_str class must be strings'), 'jyear_str': TypeError('Input values for jyear_str class must be strings'), 'astropy_time': TypeError('Input values for astropy_time class must all be same astropy Time type.')}

In [ ]:
%matplotlib inline

fig=plt.figure(1, figsize=(19.2,10.8), dpi=100) #full hd
ax = plt.subplot2grid((1,1), (0, 0), rowspan=7, projection='polar')

plt.plot(solo_lon,solo_r,'b')
plt.plot(psp_lon,psp_r,'k')

print(np.size(solo_lon))



## for larger arrays

### heliopy

In [ ]:
#heliopy - not for Solar Orbiter because this uses an old kernel


def cart2sphere(x,y,z):
    r = np.sqrt(x**2+ y**2 + z**2)           
    theta = np.arctan2(z,np.sqrt(x**2+ y**2))
    phi = np.arctan2(y,x)                    
    return (r, theta, phi)

starttime = datetime.datetime(2022, 9, 1)
endtime = starttime + timedelta(days=30)
psp_time2 = []
while starttime < endtime:
    psp_time2.append(starttime)
    starttime += timedelta(hours=24) 

spice.furnish(spicedata.get_kernel('psp_pred'))
psp2=spice.Trajectory('SPP')
psp2.generate_positions(psp_time2, 'Sun','HEEQ')
psp2.change_units(astropy.units.AU)
[psp2_r, psp2_lat, psp2_lon]=cart2sphere(psp2.x,psp2.y,psp2.z)


fig=plt.figure(1, figsize=(19.2,10.8), dpi=100) #full hd
ax = plt.subplot2grid((1,1), (0, 0), rowspan=7, projection='polar')

plt.plot(psp_lon,psp_r,'k')
plt.plot(psp2_lon,psp2_r,'ob',markersize=5)

In [ ]:
#print(np.nanmean(psp2_r.value[0:100]-psp_r[0:100]))

## astrospice for Solar Orbiter (note: not compatible with HelioSat!)

In [ ]:
kernels = astrospice.registry.get_kernels('solar orbiter', 'predict')
solo_kernel = kernels[0]

#res_in_days=1/(24)
res_in_days=3

starttime = datetime.datetime(2022, 9, 1)
endtime = datetime.datetime(2022, 9,30)
solo2_time = []
while starttime < endtime:
        solo2_time.append(starttime)
        starttime += timedelta(days=res_in_days)

solo2_coords = astrospice.generate_coords('Solar Orbiter', solo2_time)
solo2_coords_heeq = solo2_coords.transform_to(sunpy.coordinates.HeliographicStonyhurst())

print(np.size(solo2_coords_heeq))





solo2_lon=solo2_coords_heeq.lon.to(u.rad).value
solo2_lat=solo2_coords_heeq.lat.to(u.rad).value
solo2_r=solo2_coords_heeq.radius.to(u.au).value

In [ ]:

fig=plt.figure(1, figsize=(19.2,10.8), dpi=100) #full hd
ax = plt.subplot2grid((1,1), (0, 0), rowspan=7, projection='polar')

plt.plot(solo_lon,solo_r,'k')
plt.plot(solo2_lon, solo2_r, 'ob',markersize=5)

#plt.savefig('../lineups/position_test.pdf')


In [42]:
#solo2_lon[0:100]-solo_lon[0:100]